# 简单等权策略

In [1]:
import os, sys, argparse, logging
import pandas as pd

# 首先将framework的目录加进sys.path中，方便使用相对路径导入
framework_path = os.path.join(os.path.abspath(''), '../framework')
if framework_path not in sys.path:
    sys.path.append(framework_path)

In [2]:
# 设置logging格式和级别
logging.basicConfig(
    level=logging.INFO,
    format='%(filename)s-line%(lineno)d %(levelname)s: %(message)s'
    )

## 1. 等权算法

In [3]:
from alg.alg_base import AlgBase

# 等权算法，传入资产字典和总权重
class EqualWeightAlg(AlgBase):
    def __init__(self, name) -> None:
        super().__init__(name)

    def run(self, asset_dict, weight_sum, option=1):
        logging.debug('EqualWeightAlg: run')
        return {asset: option*weight_sum/len(asset_dict) for asset in asset_dict}


In [4]:
# 测试算法

# 创建一个dummy的数据集
from component.asset.asset import Asset
dummy_dict = {i: Asset(str(i)) for i in range(3)}

ewa = EqualWeightAlg('equal_weight')
print(ewa.run(dummy_dict, 0.6))

alg_base.py-line19 INFO: alg equal_weight: init args


init YamlSvc
init DateSvc
{0: 0.19999999999999998, 1: 0.19999999999999998, 2: 0.19999999999999998}


## 2. 创建单期策略

In [5]:
from import_func import getSvc
raw_data_svc = getSvc('LxwWinddbRawDataSvc')
date_svc = getSvc('DateSvc')
date_svc.setTradeDays(raw_data_svc.getTradeDays())

from strategy.strategy_base import StrategyBase

from component.position_manager.group_position_manager import GroupPositionManager
from component.position_manager.asset_position_manager import AssetPositionManager

class MyStrategy(StrategyBase):
    def __init__(self, name, args={}) -> None:
        # self.equal_weight_option = 1
        super().__init__(name, args)

    # 设置需要使用的算法
    def _initAlgDict(self):
        self._alg_dict['equal_weight'] = EqualWeightAlg('equal_weight')

    # 设置数据集
    def _initDataset(self):
        # 父类方法初始化数据集并添加cash资产
        super()._initDataset(init_position_manager=True)

        a_000001 = Asset('000001.SH')
        a_000001.setRawNavData(raw_data_svc.getNav('aindexeodprices', '000001.SH'))

        a_000003 = Asset('000003.SH')
        a_000003.setRawNavData(raw_data_svc.getNav('aindexeodprices', '000003.SH'))

        self.getDataset().addChildAsset(a_000001)
        self.getDataset().addChildAsset(a_000003)

        # 如果进行回测，需要为每个资产初始化一个仓位管理器
        a_000001.setPositionManager(AssetPositionManager())
        a_000003.setPositionManager(AssetPositionManager())

        a_000001.setTransectionRate(2e-4)
        a_000003.setTransectionRate(2e-4)

        a_000001.setMarginRatio(0.1)
        a_000003.setMarginRatio(0.1)
        

    def run(self, id_date):
        logging.debug('{}: MyStrategy: run'.format(id_date))
        # 设置数据集的id_date
        self.setIdDate(id_date, 252)

        # 构造算法要求的输入并执行算法
        alg_input = self.getDataset().getAllAsset(ignore_cash=True, id_date=id_date)
        return self._alg_dict['equal_weight'].run(alg_input, 0.5, option=self.equal_weight_option), []


init YamlSvc
init LxwWinddbRawDataSvc
init MysqlDbConnectorSvc


date_svc.py-line31 INFO: DateSvc: set trade days


init ConstantSvc


In [6]:
from datetime import datetime

# 测试单期策略
my_strategy = MyStrategy('test', args={'equal_weight_option': -1})

my_strategy._dataset.print()
print(my_strategy.run(datetime(2020, 1, 5)))

alg_base.py-line19 INFO: alg test: init args
alg_base.py-line19 INFO: alg equal_weight: init args
strategy_base.py-line49 INFO: strategy test: init dataset


group: root
	asset: cash
	asset: 000001.SH
	asset: 000003.SH
({'000001.SH': -0.25, '000003.SH': -0.25}, [])


# 3. 执行回测

In [7]:
from backtest_manager.backtest_manager import BackTestManager

backtest_args = {
    'date_range': [[2010, 1, 1], [2010, 12, 31]], 
    'frequency': 'monthly', 
    'cash': 1e4, 
    }

strategy_args = {
    'equal_weight_option': 1
}

my_backtest_manager = BackTestManager(
    MyStrategy('test', args=strategy_args), # 策略实例
    name = 'equal_weight_backtest', 
    args = backtest_args, 
)

my_backtest_manager.backtest()

alg_base.py-line19 INFO: alg test: init args
alg_base.py-line19 INFO: alg equal_weight: init args
strategy_base.py-line49 INFO: strategy test: init dataset
backtest_manager.py-line26 INFO: init BackTestManager
backtest_manager.py-line61 INFO: setting date index
backtest_manager.py-line68 INFO: setting init cash
strategy_base.py-line64 INFO: test: init cash
backtest: 100%|██████████| 242/242 [00:04<00:00, 59.17days/s]


In [8]:
# 检查回测结果

# 策略历史仓位
print('*' * 50)
print('strategy: ')
print(my_backtest_manager.getDataset().getPositionManager().getHistoricalData())

# 现金历史仓位
print('*' * 50)
print('cash: ')
print(my_backtest_manager.getDataset().getAsset('cash').getPositionManager().getHistoricalData())

# 000001.SH历史仓位
print('*' * 50)
print('000001.SH: ')
print(my_backtest_manager.getDataset().getAsset('000001.SH').getPositionManager().getHistoricalData())

# # 历史订单
print('*' * 50)
print('orders: ')
print(my_backtest_manager.getOrderManager().getAllOrders())

**************************************************
strategy: 
                position  truth_position  cost_per_unit  weight  truth_weight  \
2010-01-04  14499.856652    10000.490830            0.0     1.0           1.0   
2010-01-05  14563.458032    10064.092209            0.0     1.0           1.0   
2010-01-06  14535.150638    10035.784816            0.0     1.0           1.0   
2010-01-07  14450.003370     9950.637548            0.0     1.0           1.0   
2010-01-08  14482.375063     9983.009240            0.0     1.0           1.0   
...                  ...             ...            ...     ...           ...   
2010-12-27  18685.989191    10765.028532            0.0     1.0           1.0   
2010-12-28  18551.631223    10630.670564            0.0     1.0           1.0   
2010-12-29  18613.574654    10692.613995            0.0     1.0           1.0   
2010-12-30  18646.214758    10725.254099            0.0     1.0           1.0   
2010-12-31  18824.299969    10903.339310       